In [28]:
import mido
from mido import MidiTrack, Message, MidiFile, MidiFile

In [38]:
mid = mido.MidiFile()
track = MidiTrack()
track2 = MidiTrack()
mid.tracks.append(track)
mid.tracks.append(track2)

track.append(Message('program_change', program=12, time=0))
track2.append(Message('program_change', program=12, time=0))

# 音符开始
track.append(Message('note_on', note=64, velocity=64, time=32))
track2.append(Message('note_on', note=64, velocity=64, time=256))

# 音符结束
track.append(Message('note_off', note=64, velocity=127, time=32))
# track2.append(Message('program_change', program=12, time=64))

mid.save("new_song.mid")
from midi2audio import FluidSynth

# https://musical-artifacts.com/artifacts/855
fs = FluidSynth(sound_font='./lab/SGM-v2.01-NicePianosGuitarsBass-V1.2.sf2')  # 指定 SoundFont 文件
fs.midi_to_audio('new_song.mid', 'output.wav')
import IPython

IPython.display.Audio("output.wav")

FluidSynth runtime version 2.4.1
Copyright (C) 2000-2024 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'output.wav'..


fluidsynth: warning: End of the MIDI file reached, but not all notes have received a note off event! OFFing them now! Run with --verbose to spot pending voices.


AttributeError: 'MidiFile' object has no attribute 'tps'

In [31]:
import mido

midi = mido.MidiFile("new_song.mid")

# sequence = []
for track in midi.tracks:
    for msg in track:
        print(msg)

program_change channel=0 program=12 time=0
note_on channel=0 note=64 velocity=64 time=32
note_off channel=0 note=64 velocity=127 time=32
MetaMessage('end_of_track', time=0)
program_change channel=0 program=12 time=0
note_on channel=0 note=64 velocity=64 time=256
MetaMessage('end_of_track', time=0)


In [47]:
def msg_to_token(single_msg):
    msg_tokens = []
    if single_msg.time > 0:
        msg_tokens.append(f"Wait_{single_msg.time}")
    if single_msg.is_meta:
        pass
    elif single_msg.type == "note_on" and single_msg.velocity > 0:
        msg_tokens.append(f"Note_On_{single_msg.note}")
        msg_tokens.append(f"Velocity_{single_msg.velocity}")
    elif single_msg.type == "note_off" or (
            single_msg.type == "note_on" and single_msg.velocity == 0
    ):
        msg_tokens.append(f"Note_Off_{single_msg.note}")
    elif single_msg.type == "control_change":
        msg_tokens.append(f"Control_{single_msg.control}_{single_msg.value}")
    elif single_msg.type == "program_change":
        msg_tokens.append(f"Program_{single_msg.program}")
    elif single_msg.type == "time_signature":
        msg_tokens.append(f"Time_Signature_{single_msg.numerator}/{single_msg.denominator}")
    elif single_msg.type == "set_tempo":
        msg_tokens.append(f"Tempo_{single_msg.tempo}")
    elif single_msg.type == "sysex":
        # 跳过sysex
        pass
    elif single_msg.type == "pitchwheel":
        # print(msg)
        msg_tokens.append(f"Pitchwheel_{single_msg.pitch}")
    else:
        raise Exception(f"Unknown message type: {single_msg.type}")
    return msg_tokens

def tokenize(midi):
    final = []
    final.append(f"TicksPerBeat{midi.ticks_per_beat}")
    for i, track in enumerate(midi.tracks):
        final.append(f"StartTrack_{i}")
        for msg in track:
            for token in msg_to_token(msg):
                final.append(token)
        final.append(f"EndTrack_{i}")
    return final


tokenize(midi)
# print(midi.ticks_per_beat)

['TicksPerBeat480',
 'StartTrack_0',
 'Program_12',
 'Wait_32',
 'Note_On_64',
 'Velocity_64',
 'Wait_32',
 'Note_Off_64',
 'EndTrack_0',
 'StartTrack_1',
 'Program_12',
 'Wait_256',
 'Note_On_64',
 'Velocity_64',
 'EndTrack_1']

In [34]:
real = MidiFile("./dataset/nesmdb/nesmdb_midi/nesmdb_midi/train/000_10_YardFight_00_01GameStart.mid")
tokenize(real)

['Program_80',
 'Control_12_3',
 'Note_On_65',
 'Velocity_15',
 'Control_11_14',
 'Control_11_15',
 'Control_11_14',
 'Control_11_13',
 'Control_11_12',
 'Control_11_11',
 'Control_11_10',
 'Control_11_9',
 'Note_Off_65',
 'Note_On_69',
 'Velocity_9',
 'Control_11_15',
 'Control_11_14',
 'Control_11_13',
 'Note_On_65',
 'Velocity_13',
 'Note_Off_69',
 'Control_11_15',
 'Control_11_14',
 'Control_11_13',
 'Note_Off_65',
 'Note_On_69',
 'Velocity_13',
 'Control_11_15',
 'Control_11_14',
 'Control_11_13',
 'Note_Off_69',
 'Note_On_70',
 'Velocity_13',
 'Control_11_15',
 'Control_11_14',
 'Control_11_13',
 'Control_11_15',
 'Control_11_14',
 'Control_11_13',
 'Control_11_12',
 'Control_11_11',
 'Control_11_10',
 'Control_11_9',
 'Note_Off_70',
 'Note_On_74',
 'Velocity_9',
 'Control_11_15',
 'Control_11_14',
 'Control_11_13',
 'Note_On_70',
 'Velocity_13',
 'Note_Off_74',
 'Control_11_15',
 'Control_11_14',
 'Control_11_13',
 'Note_Off_70',
 'Note_On_74',
 'Velocity_13',
 'Control_11_15',
